In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
true_b = 1 # parameter setting
true_w = 2 #true parameter
N = 100 #n_size

# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1) #random x number n size
epsilon = (.1 * np.random.randn(N, 1)) # error term 1/10 of the true number..?
y = true_b + true_w * x + epsilon 


# Shuffles the indices
idx = np.arange(N) 
np.random.shuffle(idx) #shuffled indices

# Uses first 80 random indices for train
train_idx = idx[:int(N*.8)] #select first 80% as training
# Uses the remaining indices for validation
val_idx = idx[int(N*.8):] #select the last 20% as validation

#print(train_idx, val_idx)

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx] # training index are used to retrieve datapoint
x_val, y_val = x[val_idx], y[val_idx] #training index are used to retrieve datapoints.


x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

y_train_tensor.type()

'torch.FloatTensor'

Use Parameter class to initialize the model parameter

https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#parameter


In [11]:

class ManualLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float, device = device))
    self.w = nn.Parameter(torch.randn(1, requires_grad=True, dtype = torch.float, device= device))

  def forward(self, x):
    return self.b + self.w * x



Then this constructed parameter can be accessed using [.parameters()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters)


In [12]:
torch.manual_seed(42) # Creates a "dummy" instance of our ManualLinearRegression model 
dummy = ManualLinearRegression() 
list(dummy.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True), Parameter containing:
 tensor([0.1288], requires_grad=True)]

We can get the current state of the parameter;
https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.state_dict


In [4]:
dummy.state_dict()

OrderedDict([('b', tensor([0.3367])), ('w', tensor([0.1288]))])

In [14]:
torch.manual_seed(42)
dummy = ManualLinearRegression().to(device)

now we use the model.

In [15]:
lr = 0.1

torch.manual_seed(42)
model = ManualLinearRegression().to(device)

optimizer = optim.SGD(model.parameters(), lr = lr) #here we pass on model.parameters.
loss_fn = nn.MSELoss(reduction = 'mean')

n_epochs = 1000

for epoch in range(n_epochs):
  model.train()

  yhat = model(x_train_tensor)
  loss = loss_fn(yhat, y_train_tensor)

  loss.backward()

  optimizer.step()
  optimizer.zero_grad()

print(model.state_dict())

OrderedDict([('b', tensor([1.0235])), ('w', tensor([1.9690]))])
